<h1>The BurnMan Tutorial</h1>
<h2>Part 4: Fitting</h2>
This file is part of BurnMan - a thermoelastic and thermodynamic toolkit
for the Earth and Planetary Sciences

Copyright (C) 2012 - 2021 by the BurnMan team,
released under the GNU GPL v2 or later.


# Introduction

This ipython notebook is the fourth in a series designed to introduce new users to the code structure and functionalities present in BurnMan.

## Demonstrates

1. burnman.


Everything in BurnMan and in this tutorial is defined in SI units. 

## Importing BurnMan

In the first part of this tutorial, we decided whether to install the version of BurnMan associated with the tutorial or use the local version. If you chose to install globally, or if you want to use a previously installed version, change the "use_installed" variable below to "True".

In [ ]:
use_installed = False

if not use_installed:
    import os
    import sys
    if not os.path.exists('burnman') and os.path.exists('../../burnman'):
        sys.path.insert(1, os.path.abspath('../../'))

import burnman
print(f'This tutorial will use BurnMan version {burnman.__version__}.')

# Fitting

## Fitting to an equation of state

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from burnman.tools.unitcell import molar_volume_from_unit_cell_volume
from burnman.tools.misc import attribute_function
from burnman.optimize.nonlinear_fitting import confidence_prediction_bands

print('Least squares equation of state fitting\n')

print('1) Fitting to room temperature PV data\n')
# Now let's fit some EoS data
# Let's just take a bit of data from Andrault et al. (2003) on stishovite
PV = np.array([[0.0001, 46.5126, 0.0061],
               [1.168, 46.3429, 0.0053],
               [2.299, 46.1756, 0.0043],
               [3.137, 46.0550, 0.0051],
               [4.252, 45.8969, 0.0045],
               [5.037, 45.7902, 0.0053],
               [5.851, 45.6721, 0.0038],
               [6.613, 45.5715, 0.0050],
               [7.504, 45.4536, 0.0041],
               [8.264, 45.3609, 0.0056],
               [9.635, 45.1885, 0.0042],
               [11.69, 44.947, 0.002],                
               [17.67, 44.264, 0.002],
               [22.38, 43.776, 0.003],
               [29.38, 43.073, 0.009],
               [37.71, 42.278, 0.008],
               [46.03, 41.544, 0.017],
               [52.73, 40.999, 0.009],
               [26.32, 43.164, 0.006],
               [30.98, 42.772, 0.005],
               [34.21, 42.407, 0.003],
               [38.45, 42.093, 0.004],
               [43.37, 41.610, 0.004],
               [47.49, 41.280, 0.007]])

PTV = np.array([PV.T[0]*1.e9,
                PV.T[0]*0. + 298.15,
                molar_volume_from_unit_cell_volume(PV.T[1], 2.)]).T

nul = 0.*PTV.T[0]
PTV_covariances = np.array([[0.03*PTV.T[0], nul, nul],
                            [nul, nul, nul],
                            [nul, nul, molar_volume_from_unit_cell_volume(PV.T[2], 2.)]]).T

# Here's where we fit the data
# The mineral parameters are automatically updated during fitting
stv = burnman.minerals.HP_2011_ds62.stv()
params = ['V_0', 'K_0', 'Kprime_0']
fitted_eos = burnman.eos_fitting.fit_PTV_data(stv, params, PTV, PTV_covariances, verbose=False)

# Print the optimized parameters
print('Optimized equation of state for stishovite:')
burnman.tools.misc.pretty_print_values(fitted_eos.popt, fitted_eos.pcov, fitted_eos.fit_params)
print('')

# Create a corner plot of the covariances
fig = burnman.nonlinear_fitting.corner_plot(fitted_eos.popt, fitted_eos.pcov,
                                            params)
axes = fig[1]
axes[1][0].set_xlabel('$V_0$ ($\\times 10^{-5}$ m$^3$)')
axes[1][1].set_xlabel('$K_0$ ($\\times 10^{11}$ Pa)')
axes[0][0].set_ylabel('$K_0$ ($\\times 10^{11}$ Pa)')
axes[1][0].set_ylabel('$K\'_0$')
plt.show()

# Finally, let's plot our equation of state
T = 298.15
pressures = np.linspace(1.e5, 60.e9, 101)
volumes = np.empty_like(pressures)

PTVs = np.empty((len(pressures), 3))
for i, P in enumerate(pressures):
    stv.set_state(P, T)
    PTVs[i] = [P, T, stv.V]

# Plot the 95% confidence and prediction bands
cp_bands = confidence_prediction_bands(model = fitted_eos,
                                       x_array = PTVs,
                                       confidence_interval = 0.95,
                                       f=attribute_function(stv, 'V'),
                                       flag='V')

plt.fill_between(PTVs[:,0]/1.e9, cp_bands[2] * 1.e6, cp_bands[3] * 1.e6,
                 color=[0.75, 0.25, 0.55], label='95% prediction bands')
plt.fill_between(PTVs[:,0]/1.e9, cp_bands[0] * 1.e6, cp_bands[1] * 1.e6,
                 color=[0.75, 0.95, 0.95], label='95% confidence bands')

plt.plot(PTVs[:,0] / 1.e9, PTVs[:,2] * 1.e6,
          label='Optimized fit for stishovite')
plt.errorbar(PTV[:,0] / 1.e9, PTV[:,2] * 1.e6,
              xerr=PTV_covariances.T[0][0] / 1.e9,
              yerr=PTV_covariances.T[2][2] * 1.e6,
              linestyle='None', marker='o', label='Andrault et al. (2003)')

plt.ylabel("Volume (cm$^3$/mol)")
plt.xlabel("Pressure (GPa)")
plt.legend(loc="upper right")
plt.title("Stishovite EoS (room temperature)")
plt.show()


# Plot the 95% confidence and prediction bands for the bulk modulus
cp_bands = confidence_prediction_bands(model = fitted_eos,
                                       x_array = PTVs,
                                       confidence_interval = 0.95,
                                       f=attribute_function(stv, 'K_T'),
                                       flag='V')

plt.fill_between(PTVs[:,0]/1.e9, (cp_bands[0])/1.e9, (cp_bands[1])/1.e9, color=[0.75, 0.95, 0.95], label='95% confidence band')
plt.plot(PTVs[:,0]/1.e9, (cp_bands[0] + cp_bands[1])/2.e9, color='b', label='Best fit')

plt.ylabel("Bulk modulus (GPa)")
plt.xlabel("Pressure (GPa)")
plt.legend(loc="upper right")
plt.title("Stishovite EoS; uncertainty in bulk modulus (room temperature)")
plt.show()

## Fitting to a compositional model

In [ ]:
    from burnman import minerals
    from burnman.optimize.composition_fitting import fit_composition_to_solution
    
    # Here, we use the Jennings and Holland (2015) garnet,
    # which has five endmembers: pyrope, almandine, grossular,
    # andradite and knorringite
    gt = minerals.JH_2015.garnet()

    print(f'Endmembers: {gt.endmember_names}')
    print(f'Elements: {gt.elements}')
    print('Stoichiometric matrix:')
    print(gt.stoichiometric_matrix)
    print()

    # The variables we are trying to fit are Fe (total), Ca, Mg
    # Cr, Al, Si and Fe3+, all given in mole amounts.
    fitted_variables = ['Fe', 'Ca', 'Mg', 'Cr', 'Al', 'Si', 'Fe3+']
    variable_values = np.array([1.1, 2., 0., 0, 1.9, 3., 0.1])
    variable_covariances = np.eye(7)*0.01*0.01

    # Add some noise.
    v_err = np.random.rand(7)
    np.random.seed(100)
    variable_values = np.random.multivariate_normal(variable_values,
                                                    variable_covariances)

    # Fe3+ isn't an element or a site-species of the solution model,
    # so we need to provide the linear conversion from Fe3+ to
    # elements and/or site species. In this case, Fe3+ resides only on
    # the second site, and the JH_2015.gt model has labelled Fe3+ on that
    # site as Fef. Therefore, the conversion is simply Fe3+ = Fef_B.
    variable_conversions = {'Fe3+': {'Fef_B': 1.}}

    # The next line does the heavy lifting
    popt, pcov, res = fit_composition_to_solution(gt,
                                                  fitted_variables,
                                                  variable_values,
                                                  variable_covariances,
                                                  variable_conversions)

    # We can set the composition of gt using the optimized parameters
    gt.set_composition(popt)

    # Print the optimized parameters and principal uncertainties
    print('Molar fractions:')
    for i in range(len(popt)):
        print(f'{gt.endmember_names[i]}: '
              f'{gt.molar_fractions[i]:.3f} +/- '
              f'{np.sqrt(pcov[i][i]):.3f}')

    print(f'Weighted residual: {res:.3f}')

## Fitting phase proportions